# Notebook phuc hoi anh co nhieu

Do tác động của Degration function nên chất lượng ảnh bị suy giảm. <br>
Trong notebook này, chúng tôi thực hiện qua 2 quá trình <br>
<ol>
<li> Tạo nhiễu cho ảnh gốc
<li> Phục hồi ảnh nhiễu thông qua các bộ lọc phù hợp
</ol>

## Inverse Filter

Đầu tiên, thêm nhiễu vào ảnh. <br>



In [ ]:
from add_noise import SaltAndPeperNoise
import cv2
from config import Flag
import matplotlib.pyplot as plt


prefix = Flag.image_origin
file_name = prefix / 'banana.jpg'

saltAndPeperNoise = SaltAndPeperNoise(str(file_name), s_vs_p=1, amount=0.1)
sapn = saltAndPeperNoise.noise()
# print(sapn)
image_noise = sapn
plt.imshow(sapn)

## Wiener Filter